This script computes compactness measures
(https://fisherzachary.github.io/public/r-output.html)


In [22]:
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
f = r'Q:\projects\GeoBC\Human Disturbance\data\Landclass\NRcan_urban_cleanup\urban_sieve_60px_4con_poly.shp'

gdf = gpd.read_file(f)

In [ ]:
def pp_compactness(geom):
    """Polsby‑Popper score."""
    p = geom.length
    a = geom.area    
    return (4 * math.pi * a) / (p * p)

def s_compactness(geom):
    """Schwartzberg score."""
    p = geom.length
    a = geom.area    
    # circumference of circle with area a:
    c = 2 * math.pi * math.sqrt(a / math.pi)
    return 1 / (p / c)

def ch_compactness(geom):
    """Convex Hull score: area district / area convex hull."""
    a = geom.area
    a_ch = geom.convex_hull.area
    return a / a_ch if a_ch > 0 else None


def mbr_dimensions(geom):
    """
    Returns (length, width) of the minimum rotated rectangle.
    """
    mbr = geom.minimum_rotated_rectangle
    coords = list(mbr.exterior.coords)
    # compute all edge lengths
    edges = [
        Point(coords[i]).distance(Point(coords[i+1]))
        for i in range(len(coords) - 1)
    ]
    if len(edges) < 2:
        return None, None
    length = max(edges)
    width  = min(edges)
    return length, width

# expand into two columns
gdf[['minRec_l', 'minRec_w']] = gdf.geometry.apply(
    lambda g: pd.Series(mbr_dimensions(g))
)


def rect_compactness(geom):
    """Rectangularity: Area ÷ (Lengthₘₐₓ × Widthₘᵢₙ) of MBR"""
    mbr = geom.minimum_rotated_rectangle
    coords = list(mbr.exterior.coords)
    edges = [Point(coords[i]).distance(Point(coords[i+1]))
             for i in range(len(coords)-1)]
    if len(edges) < 2:
        return None
    L, W = max(edges), min(edges)
    return geom.area / (L * W) if L * W > 0 else None



# apply to your GeoDataFrame
gdf["c_pp"]    = gdf.geometry.apply(pp_compactness)
gdf["c_schw"] = gdf.geometry.apply(s_compactness)
gdf["c_ch"]   = gdf.geometry.apply(ch_compactness)
gdf['c_lw'] = gdf['minRec_w'] / gdf['minRec_l']

gdf["c_rect"]= gdf.geometry.apply(rect_compactness)


# standardize measures to [0,1]
metrics = gdf[["c_lw", "minRec_w"]].fillna(0)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(metrics)

# Option A: composite via geometric mean of normalized metrics
gdf["shape_idx"] = scaled.prod(axis=1) ** (1/4)

# Option B: composite via first principal component
pca = PCA(n_components=1)
gdf["pc1_idx"] = pca.fit_transform(scaled)


In [ ]:
gdf.to_file(r'Q:\projects\GeoBC\Human Disturbance\data\Landclass\NRcan_urban_cleanup\urban_sieve_60px_4con_poly_compactnessAttr.shp', driver='ESRI Shapefile')